# Sparse Tensors

```
Copyright 2022 National Technology & Engineering Solutions of Sandia,
LLC (NTESS). Under the terms of Contract DE-NA0003525 with NTESS, the
U.S. Government retains certain rights in this software.
```

The Sptensor class stores the data in coordinate format.

## Contents

* [Creating a Sptensor](#Creating-a-Sptensor)
* [Specifying the accumulation method for the constructor](#Specifying-the-accumulation-method-for-the-constructor)
* [Creating a one-dimensional Sptensor](#Creating-a-one-dimensional-Sptensor)
* [Creating an all-zero Sptensor](#Creating-an-all-zero-Sptensor)
* [Constituent parts of a Sptensor](#Constituent-parts-of-a-Sptensor)
* [Creating a sparse tensor from its constituent parts](#Creating-a-sparse-tensor-from-its-constituent-parts)
* [Creating an empty Sptensor](#Creating-an-empty-Sptensor)
* [Use sptenrand to create a random Sptensor](#Use-sptenrand-to-create-a-random-Sptensor)
* [Use squeeze to remove singleton dimensions from a Sptensor](#Use-squeeze-to-remove-singleton-dimensions-from-a-Sptensor)
* [Use squash to remove empty slices from a Sptensor](#Use-squash-to-remove-empty-slices-from-a-Sptensor)
* [Use full or tensor to convert a sptensor to a (dense) tensor](#Use-full-or-tensor-to-convert-a-sptensor-to-a-(dense)-tensor)
* [Use Sptensor to convert a (dense) tensor to a Sptensor](#Use-Sptensor-to-convert-a-(dense)-tensor-to-a-Sptensor)
* [Use double to convert a sptensor to a (dense) multidimensional array](#Use-double-to-convert-a-sptensor-to-a-(dense)-multidimensional-array)
* [Use find to extract nonzeros from a tensor and then create a Sptensor](#Use-find-to-extract-nonzeros-from-a-tensor-and-then-create-a-Sptensor)
* [Use ndims and size to get the size of a Sptensor](#Use-ndims-and-size-to-get-the-size-of-a-Sptensor)
* [Use nnz to get the number of nonzeroes of a Sptensor](#Use-nnz-to-get-the-number-of-nonzeroes-of-a-Sptensor)
* [Subscripted reference for a Sptensor](#Subscripted-reference-for-a-Sptensor)
* [Subscripted assignment for a Sptensor](#Subscripted-assignment-for-a-Sptensor)
* [**BREAK**](#BREAK)
* [Nonzero reference for a Sptensor](#Nonzero-reference-for-a-Sptensor)
* [Nonzero assignment for a Sptensor](#Nonzero-assignment-for-a-Sptensor)

* [Basic operations on a Sptensor](#Basic-operations-on-a-Sptensor)
     * [Addition](#Addition)
     * [Subtraction](#Subtraction)
     * [Multiplication](#Multiplication)
     * [Division](#Division)
     * [Power](#Power)
     * [Equality](#Equality)
* [Displaying a Sptensor](#Displaying-a-Sptensor)

## Creating a Sptensor

A sparse tensor can be created by passing in a list of subscripts and values. For example, here we pass in three subscripts and a scalar value. The resuling sparse tensor has three nonzero entries, and the shape is the size of the largest subscript in each dimension.

In [1]:
import pyttb as ttb
import numpy as np

In [2]:
np.random.seed(0)
subs =  np.array([[0,0,0],[0,1,0],[2,3,1]]) # Subscripts of the nonzeros.
vals = np.array([[1],[2],[3]]) # Vals is a column vector; values of the nonzeros.
X = ttb.sptensor.from_aggregator(subs,vals) # Sparse tensor with 3 nonzeros.

In [3]:
X = ttb.sptensor.from_data(subs,vals,(3, 5, 2))   # Or, specify the shape explicitly.
X

Sparse tensor of shape 3 x 5 x 2 with 3 nonzeros 
	[0, 0, 0] = 1
	[0, 1, 0] = 2
	[2, 3, 1] = 3

Values corresponding to repeated subscripts are summed.

In [4]:
subs = np.array([[0,0,0],[0,0,2],[2,2,2],[3,3,3],[0,0,0],[0,0,0]])  # (1,1,1) is repeated.
vals = np.array([2,2,2,2,2,2])[:, None]  # Vals is a column vector.
X = ttb.sptensor.from_aggregator(subs,vals)
X

Sparse tensor of shape 4 x 4 x 4 with 4 nonzeros 
	[0, 0, 0] = 6
	[0, 0, 2] = 2
	[2, 2, 2] = 2
	[3, 3, 3] = 2

## Specifying the accumulation method for the constructor

In [5]:
subs = np.array([[0,0,0],[0,0,2],[2,2,2],[3,3,3],[0,0,0],[0,0,0]])
vals = 2 * np.ones((6, 1))  # A column vector of 2s
shape = (4, 4, 4)
X = ttb.sptensor.from_aggregator(subs, vals, shape, np.max) # Maximum element.
X

Sparse tensor of shape 4 x 4 x 4 with 4 nonzeros 
	[0, 0, 0] = 2.0
	[0, 0, 2] = 2.0
	[2, 2, 2] = 2.0
	[3, 3, 3] = 2.0

In [6]:
myfun = myfun = lambda x: np.sum(x) / 3 # Total sum divided by three.
X = ttb.sptensor.from_aggregator(subs, vals, shape, myfun) # Custom accumulation function.
X

Sparse tensor of shape 4 x 4 x 4 with 4 nonzeros 
	[0, 0, 0] = 2.0
	[0, 0, 2] = 0.6666666666666666
	[2, 2, 2] = 0.6666666666666666
	[3, 3, 3] = 0.6666666666666666

## Creating a one-dimensional Sptensor

In [7]:
X = ttb.sptensor.from_aggregator(np.array([[0],[2],[4]]),np.ones((3,1)))
X

Sparse tensor of shape 5 with 3 nonzeros 
	[0] = 1.0
	[2] = 1.0
	[4] = 1.0

In [8]:
np.random.seed(0)
X = ttb.sptenrand((50,), nonzeros=5)
X

Sparse tensor of shape 50 with 5 nonzeros 
	[19] = 0.7917250380826646
	[21] = 0.5288949197529045
	[32] = 0.5680445610939323
	[44] = 0.925596638292661
	[48] = 0.07103605819788694

## Creating an all-zero Sptensor

In [9]:
X = ttb.sptensor()
X[9,9,9] = 0 # Creates an all-zero tensor.
X

All-zero sparse tensor of shape 10 x 10 x 10

## Constituent parts of a Sptensor

In [10]:
np.random.seed(0)
X = ttb.sptenrand([40, 30, 20], nonzeros=5) # Create data.
X.subs # Subscripts of nonzeros.

array([[15, 23, 10],
       [17, 26, 19],
       [21, 12, 12],
       [21, 21, 12],
       [22, 27,  1]])

In [11]:
X.vals # Corresponding nonzero values.

array([[0.0871293 ],
       [0.0202184 ],
       [0.83261985],
       [0.77815675],
       [0.87001215]])

In [12]:
X.shape # The shape.

[40, 30, 20]

## Creating a sparse tensor from its constituent parts

In [13]:
np.random.seed(0)
X = ttb.sptenrand([40, 30, 20], nonzeros=5) # Create data.
Y = ttb.sptensor.from_tensor_type(X)
Y

Sparse tensor of shape 40 x 30 x 20 with 5 nonzeros 
	[15, 23, 10] = 0.08712929970154071
	[17, 26, 19] = 0.02021839744032572
	[21, 12, 12] = 0.832619845547938
	[21, 21, 12] = 0.7781567509498505
	[22, 27, 1] = 0.8700121482468192

## Creating an empty Sptensor

In [14]:
Y = ttb.sptensor() # Create an empty sptensor.
Y

All-zero sparse tensor of shape ()

## Use sptenrand to create a random Sptensor

In [15]:
np.random.seed(0)
X = ttb.sptenrand([10, 10, 10], 0.01) # Create a tesnor with 1% nonzeros using the 'density' param.
X

Sparse tensor of shape 10 x 10 x 10 with 10 nonzeros 
	[0, 0, 8] = 0.26455561210462697
	[1, 6, 1] = 0.7742336894342167
	[3, 7, 5] = 0.45615033221654855
	[4, 8, 9] = 0.5684339488686485
	[5, 4, 6] = 0.018789800436355142
	[5, 7, 6] = 0.6176354970758771
	[5, 9, 0] = 0.6120957227224214
	[7, 4, 7] = 0.6169339968747569
	[7, 8, 9] = 0.9437480785146242
	[9, 5, 4] = 0.6818202991034834

In [16]:
np.random.seed(0)
X = ttb.sptenrand([10, 10, 10], nonzeros=10) # Create a tensor with 10 nonzeros using the 'nonzeros' param.
X

Sparse tensor of shape 10 x 10 x 10 with 10 nonzeros 
	[0, 0, 8] = 0.26455561210462697
	[1, 6, 1] = 0.7742336894342167
	[3, 7, 5] = 0.45615033221654855
	[4, 8, 9] = 0.5684339488686485
	[5, 4, 6] = 0.018789800436355142
	[5, 7, 6] = 0.6176354970758771
	[5, 9, 0] = 0.6120957227224214
	[7, 4, 7] = 0.6169339968747569
	[7, 8, 9] = 0.9437480785146242
	[9, 5, 4] = 0.6818202991034834

## Use squeeze to remove singleton dimensions from a Sptensor

In [17]:
indices = np.array([[0, 0, 0], [1, 0, 0]])
values = np.ones((2,))

Y = ttb.sptensor.from_aggregator(indices, values) # Create a sparse tensor.
Y

Sparse tensor of shape 2 x 1 x 1 with 2 nonzeros 
	[0, 0, 0] = 1.0
	[1, 0, 0] = 1.0

In [18]:
Y.squeeze() # Remove singleton dimensions.

Sparse tensor of shape 2 with 2 nonzeros 
	[0] = 1.0
	[1] = 1.0

## Use squash to remove empty slices from a Sptensor

In [19]:
indices = np.array([[0, 0, 0], [2, 2, 2]])
values = np.array([1,3])

Y = ttb.sptensor.from_aggregator(indices, values) # Create a sparse tensor.
Y

Sparse tensor of shape 3 x 3 x 3 with 2 nonzeros 
	[0, 0, 0] = 1
	[2, 2, 2] = 3

## TODO: SQUASH IS not implemented

In [20]:
# Y.squash()

## Use full or tensor to convert a sptensor to a (dense) tensor

In [21]:
indices = np.array([[0, 0, 0], [1, 1, 1]])
values = np.array([[1], [1]])
X = ttb.sptensor.from_aggregator(indices, values) # Create a sparse tensor.
X.full() # Convert it to a (dense) tensor.

tensor of shape 2 x 2 x 2
data[0, :, :] = 
[[1. 0.]
 [0. 0.]]
data[1, :, :] = 
[[0. 0.]
 [0. 1.]]

In [22]:
Y = ttb.tensor.from_tensor_type(X) # Same as above.
Y

tensor of shape 2 x 2 x 2
data[0, :, :] = 
[[1. 0.]
 [0. 0.]]
data[1, :, :] = 
[[0. 0.]
 [0. 1.]]

## Use Sptensor to convert a (dense) tensor to a Sptensor

In [23]:
indices = np.array([[0, 0, 0], [1, 1, 1]])
values = np.array([[1], [1]])
X = ttb.sptensor.from_aggregator(indices, values) # Create a sparse tensor.
Y = ttb.tensor.from_tensor_type(X) # Convert it to a (dense) tensor.
Z = ttb.sptensor.from_tensor_type(Y) # Convert a tensor to a sptensor.
Z

Sparse tensor of shape 2 x 2 x 2 with 2 nonzeros 
	[0, 0, 0] = 1.0
	[1, 1, 1] = 1.0

## Use double to convert a sptensor to a (dense) multidimensional array

In [24]:
indices = np.array([[0, 0, 0], [1, 1, 1]])
values = np.array([[1], [1]])
X = ttb.sptensor.from_aggregator(indices, values) # Create a sparse tensor.
Y = ttb.sptensor.double(X) # Creates numpy.ndarray
Y 

array([[[1., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 1.]]])

## Use find to extract nonzeros from a tensor and then create a Sptensor

In [25]:
np.random.seed(0)
X = ttb.tensor.from_data(np.random.rand(5,4,2)) # Create a tensor.
larger_entries = X > 0.9 # Extract subscipts of values greater than 0.9.
subs, vals = larger_entries.find() # Extract corresponding subscripts and values.
Y = ttb.sptensor.from_aggregator(subs, vals) # Create a new sptensor.
Y

Sparse tensor of shape 5 x 4 x 2 with 5 nonzeros 
	[1, 0, 0] = 1
	[1, 2, 1] = 1
	[2, 2, 0] = 1
	[3, 1, 1] = 1
	[4, 3, 0] = 1

## Use ndims and size to get the size of a Sptensor

In [26]:
X = ttb.sptensor.from_data(np.array([[1,1,1],[2,3,2],[3,4,1],[1,0,0]]),np.array([[3],[2],[1],[3]]), (4, 5, 3))
X

Sparse tensor of shape 4 x 5 x 3 with 4 nonzeros 
	[1, 1, 1] = 3
	[2, 3, 2] = 2
	[3, 4, 1] = 1
	[1, 0, 0] = 3

In [27]:
X.ndims # Number of dimensions or modes.

3

In [28]:
X.shape # Size of X.

(4, 5, 3)

In [29]:
X.shape[2] # Size of mode 3 of X.

3

## Use nnz to get the number of nonzeroes of a Sptensor

In [30]:
X = ttb.sptensor.from_data(np.array([[1,1,1],[2,3,2],[3,4,1],[1,0,0]]),np.array([[3],[2],[1],[3]]), (4, 5, 3))
X.nnz # Number of nonzeros of X.

4

## Subscripted reference for a Sptensor

In [72]:
X = ttb.sptensor.from_data(np.array([[3,3,3],[1,1,0],[1,2,1]]), np.array([[3],[5],[1]]), (4, 4, 4)) # Create a sptensor.
X

Sparse tensor of shape 4 x 4 x 4 with 3 nonzeros 
	[3, 3, 3] = 3
	[1, 1, 0] = 5
	[1, 2, 1] = 1

In [73]:
X[0, 1, 0].item() # Extract the (0,1,0) element, which is zero.

0

In [74]:
X[3,3,3].item()  # Extract the (3,3,3) element, which is non-zero.

3

In [75]:
X[0:2,1:4,:] # Extract the 2x3x4 subtensor.

Sparse tensor of shape 2 x 3 x 4 with 2 nonzeros 
	[1, 0, 0] = 5
	[1, 1, 1] = 1

In [83]:
X[1,1,1]

array([[0]])

In [84]:
X[1,1,0]

array([[5]])

In [86]:
X[(1,)]

AssertionError: Invalid indexing

## Subscripted assignment for a Sptensor

In [36]:
X[1,2,1] = 5
print(X[1,2,1])

[[5]]


In [37]:
print(X)   #Note that (1,2,1) is now a nonzero

Sparse tensor of shape 4 x 4 x 4 with 3 nonzeros 
	[3, 3, 3] = 3
	[1, 1, 0] = 5
	[1, 2, 1] = 5


## Nonzero reference for a Sptensor

In [71]:
X[1, 1, 1]

AssertionError: Invalid indexing

## Nonzero assignment for a Sptensor

In [39]:
X[2, 2, 2] = 3.3
print(X)

Sparse tensor of shape 4 x 4 x 4 with 4 nonzeros 
	[3, 3, 3] = 3.0
	[1, 1, 0] = 5.0
	[1, 2, 1] = 5.0
	[2, 2, 2] = 3.3


You can also modify just the value

In [40]:
X[2, 2, 2] = -44.1
print(X)

Sparse tensor of shape 4 x 4 x 4 with 4 nonzeros 
	[3, 3, 3] = 3.0
	[1, 1, 0] = 5.0
	[1, 2, 1] = 5.0
	[2, 2, 2] = -44.1


## Basic operations on a Sptensor

Sptensors support plus, minus, times, divide, power, equals, and not-equals operators.  Sptensors can use there operators with another Sptensor or a scalar (with the exception of equalities which only takes Sptensors).  All mathematical operators are elementwise operations.

### Addition

In [41]:
X = ttb.sptensor.from_data(np.array([[0,0],[1,1]]),np.array([[2],[2]]),(2,2))
print(X)

Sparse tensor of shape 2 x 2 with 2 nonzeros 
	[0, 0] = 2
	[1, 1] = 2


In [42]:
Y = ttb.sptensor.from_data(np.array([[0,0],[0,1]]),np.array([[3],[3]]),(2,2))
print(Y)

Sparse tensor of shape 2 x 2 with 2 nonzeros 
	[0, 0] = 3
	[0, 1] = 3


In [43]:
X + 1  # This addition yields dense tensor

tensor of shape 2 x 2
data[:, :] = 
[[3. 1.]
 [1. 3.]]

In [44]:
X + Y  # This addition yields sparse tensor

Sparse tensor of shape 2 x 2 with 3 nonzeros 
	[0, 0] = 5
	[0, 1] = 3
	[1, 1] = 2

In [45]:
X += 2
print(X)

tensor of shape 2 x 2
data[:, :] = 
[[4. 2.]
 [2. 4.]]



### Subtraction

In [46]:
X = ttb.sptensor.from_data(np.array([[0,0],[1,1]]),np.array([[2],[2]]),(2,2))
print(X)

Sparse tensor of shape 2 x 2 with 2 nonzeros 
	[0, 0] = 2
	[1, 1] = 2


In [47]:
Y = ttb.sptensor.from_data(np.array([[0,0],[0,1]]),np.array([[3],[3]]),(2,2))
print(Y)

Sparse tensor of shape 2 x 2 with 2 nonzeros 
	[0, 0] = 3
	[0, 1] = 3


In [48]:
X - 1

tensor of shape 2 x 2
data[:, :] = 
[[ 1. -1.]
 [-1.  1.]]

In [49]:
X - Y

Sparse tensor of shape 2 x 2 with 3 nonzeros 
	[0, 0] = -1
	[0, 1] = -3
	[1, 1] = 2

In [50]:
X -= 2
print(X)

tensor of shape 2 x 2
data[:, :] = 
[[ 0. -2.]
 [-2.  0.]]



### Multiplication

In [51]:
X = ttb.sptensor.from_data(np.array([[0,0],[1,1]]),np.array([[2],[2]]),(2,2))
print(X)

Sparse tensor of shape 2 x 2 with 2 nonzeros 
	[0, 0] = 2
	[1, 1] = 2


In [52]:
Y = ttb.sptensor.from_data(np.array([[0,0],[0,1]]),np.array([[3],[3]]),(2,2))
print(Y)

Sparse tensor of shape 2 x 2 with 2 nonzeros 
	[0, 0] = 3
	[0, 1] = 3


In [53]:
X * 2.5

Sparse tensor of shape 2 x 2 with 2 nonzeros 
	[0, 0] = 5.0
	[1, 1] = 5.0

In [54]:
X * Y

Sparse tensor of shape 2 x 2 with 1 nonzeros 
	[0, 0] = 6

In [55]:
X *= 3.3
print(X)

Sparse tensor of shape 2 x 2 with 2 nonzeros 
	[0, 0] = 6.6
	[1, 1] = 6.6


### Division

In [56]:
X = ttb.sptensor.from_data(np.array([[0,0],[1,1]]),np.array([[2],[2]]),(2,2))
print(X)

Sparse tensor of shape 2 x 2 with 2 nonzeros 
	[0, 0] = 2
	[1, 1] = 2


In [57]:
Y = ttb.sptensor.from_data(np.array([[0,0],[0,1]]),np.array([[3],[3]]),(2,2))
print(Y)

Sparse tensor of shape 2 x 2 with 2 nonzeros 
	[0, 0] = 3
	[0, 1] = 3


In [58]:
X / 3

Sparse tensor of shape 2 x 2 with 2 nonzeros 
	[0, 0] = 0.6666666666666666
	[1, 1] = 0.6666666666666666

In [59]:
X / Y

Sparse tensor of shape 2 x 2 with 4 nonzeros 
	[0, 0] = 0.6666666666666666
	[1, 0] = nan
	[1, 1] = 0.0
	[1, 0] = nan

In [60]:
X /= 4
print(X)

Sparse tensor of shape 2 x 2 with 2 nonzeros 
	[0, 0] = 0.5
	[1, 1] = 0.5


### Equality

In [61]:
X = ttb.sptensor.from_data(np.array([[0,0],[1,1]]),np.array([[2],[2]]),(2,2))
print(X)

Sparse tensor of shape 2 x 2 with 2 nonzeros 
	[0, 0] = 2
	[1, 1] = 2


In [62]:
Y = ttb.sptensor.from_data(np.array([[0,0],[0,1]]),np.array([[3],[3]]),(2,2))
print(Y)

Sparse tensor of shape 2 x 2 with 2 nonzeros 
	[0, 0] = 3
	[0, 1] = 3


In [63]:
(X == Y)

Sparse tensor of shape 2 x 2 with 1 nonzeros 
	[1, 0] = True

In [64]:
X.isequal(Y)

False

In [65]:
X != Y

Sparse tensor of shape 2 x 2 with 3 nonzeros 
	[1, 1] = True
	[0, 1] = True
	[0, 0] = True

## Displaying a Sptensor

In [66]:
print(X)

Sparse tensor of shape 2 x 2 with 2 nonzeros 
	[0, 0] = 2
	[1, 1] = 2


In [67]:
X     #In the python interface

Sparse tensor of shape 2 x 2 with 2 nonzeros 
	[0, 0] = 2
	[1, 1] = 2